In [1]:
%matplotlib inline
from tensorscaling import scale, unit_tensor, random_tensor, marginal
import numpy as np

# Tensor scaling

Scale 3x3x3 unit tensor to certain non-uniform marginals:

In [2]:
shape = [3, 3, 3]
targets = [(.5, .25, .25), (.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-4)
res

Result(success=True, iterations=92, max_dist=8.827056853796172e-05, ...)

We can also access the scaling matrices and the final scaled state:

In [3]:
print(res.gs[0], "\n")
print(res.gs[1], "\n")
print(res.gs[2])

[[-0.82957348-0.39440336j  0.15732691+0.65244798j -0.92571052-0.23631889j]
 [-0.04475721+0.32019158j -0.03419995+0.09212631j  0.09660325-0.87300188j]
 [-0.28144931-0.04526811j -0.19141941-0.65174683j -0.39610112-0.16865718j]] 

[[-0.67221894-0.33391433j  0.37654244+0.16369067j -0.34298304-0.01955992j]
 [-0.1422101 -0.02617791j -0.56065823-0.36003606j  0.49754062+0.15145035j]
 [-0.21337886-0.03033133j -0.29455401-0.06418145j -1.99342569-0.23571642j]] 

[[-0.17812839-1.34146704j  0.35007668+0.90392306j -0.08111688+0.31929977j]
 [-0.45910404+0.18104402j -0.58087116+0.50737511j  0.03104345-0.03030718j]
 [-0.30963215-0.0707316j   0.16444539-0.10039052j -0.13176291-0.06244108j]]


Let's now check that the W tensor *cannot* be scaled to uniform marginals:

In [4]:
shape = [2, 2, 2, 2]
W = np.zeros(shape)
W[1, 0, 0, 0] = W[0, 1, 0, 0] = W[0, 0, 1, 0] = W[0, 0, 0, 1] = .5
targets = [(.5, .5)] * 4

scale(W, targets, eps=1e-4, max_iterations=1000)

Result(success=False, iterations=1000, max_dist=0.5934653559719872, ...)

To see more clearly what is going on, we can set the `verbose` flag:

In [5]:
res = scale(W, targets, eps=1e-4, max_iterations=10, verbose=True)

scaling tensor of shape (2, 2, 2, 2) and type float64
target spectra:
  0: (0.5, 0.5)
  1: (0.5, 0.5)
  2: (0.5, 0.5)
  3: (0.5, 0.5)
#000: max_dist = 0.35355339 @ sys = 3
#001: max_dist = 0.47140452 @ sys = 0
#002: max_dist = 0.56568542 @ sys = 1
#003: max_dist = 0.62853936 @ sys = 2
#004: max_dist = 0.58232323 @ sys = 3
#005: max_dist = 0.59305730 @ sys = 0
#006: max_dist = 0.59545834 @ sys = 1
#007: max_dist = 0.59262283 @ sys = 2
#008: max_dist = 0.59353004 @ sys = 3
#009: max_dist = 0.59357133 @ sys = 0
#010: max_dist = 0.59340661 @ sys = 1
did not converge!


We see that at each point in the algorithm, one of the marginals has Frobenius distance $\approx 0.59$ to being uniform. Indeed, we know that the entanglement polytope of the W tensor does not include the point corresponding to uniform marginals -- see [here](https://www.entanglement-polytopes.org/four_qubits) for an interactive visualization!

# Tuples of matrices and tensors

We can just as well only prescribe the desired spectra for subsystems.
Note that prescribing two out of three marginals amounts to *operator scaling*.

In [6]:
shape = [3, 3, 3]
targets = [(.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-6)
res

Result(success=True, iterations=31, max_dist=6.538952297515564e-07, ...)

Indeed, the last two marginals are as prescribed, while the first marginal is arbitrary.

In [7]:
print(marginal(res.psi, 0).round(5), "\n")
print(marginal(res.psi, 1).round(5), "\n")
print(marginal(res.psi, 2).round(5))

[[0.35186+0.j      0.01088+0.01438j 0.01673+0.01504j]
 [0.01088-0.01438j 0.31496+0.j      0.01009+0.0016j ]
 [0.01673-0.01504j 0.01009-0.0016j  0.33318+0.j     ]] 

[[ 0.4+0.j -0. +0.j  0. -0.j]
 [-0. -0.j  0.3+0.j -0. -0.j]
 [ 0. +0.j -0. +0.j  0.3+0.j]] 

[[ 0.7+0.j  0. +0.j -0. -0.j]
 [ 0. -0.j  0.2+0.j -0. +0.j]
 [-0. +0.j -0. -0.j  0.1+0.j]]
